In [154]:
import torch
from PetDataset import PetDataset
import torchvision
from PIL import Image
import os
import torchvision.transforms as transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
PSIZE=128

cuda:0


In [155]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((PSIZE, PSIZE)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = PetDataset(data_transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4)

dataiter = iter(dataloader)

images, labels = dataiter.next()

labels


tensor([0.6300, 0.4200, 0.2800, 0.1500], dtype=torch.float64)

In [156]:
def out_size(h,k, p=0,dd=1,s=1):
    return int((h+2*p-dd*(k-1)-1)/s + 1)
a=out_size(32, 5)
print(a)
b=out_size(a, 2, 0, 1, 2)
print(b)
c=out_size(b, 5)
print(c)

28
14
10


In [157]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.a = out_size(PSIZE, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.b = out_size(self.a, 2, 0, 1, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.c = int(out_size(self.b, 5)/1)
        self.fc1 = nn.Linear(16 * self.c * self.c, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * self.c * self.c)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.fc4(x)
        return x.view(-1)

In [ ]:
import torch.optim as optim
net = Net()
criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 0.052
[1,   400] loss: 0.052
[1,   600] loss: 0.050
[1,   800] loss: 0.049
[1,  1000] loss: 0.039
[1,  1200] loss: 0.044
[1,  1400] loss: 0.049
[1,  1600] loss: 0.042
[1,  1800] loss: 0.047
[1,  2000] loss: 0.040
[1,  2200] loss: 0.045
[1,  2400] loss: 0.044
[2,   200] loss: 0.042
[2,   400] loss: 0.050
[2,   600] loss: 0.047
[2,   800] loss: 0.048
[2,  1000] loss: 0.039
[2,  1200] loss: 0.043


In [ ]:
correct = 0
total = 0
iter_i = 0

with torch.no_grad():
    for data in dataloader:
        images, labels = data
        outputs = net(images)

        if iter_i == 0:
            d = outputs
        else:
            d = torch.cat((d, outputs), 0)

        iter_i += 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("petfinder-pawpularity-score/train.csv")

plt.scatter(df["Pawpularity"], d.numpy(), alpha=0.1)